 # Note
 We have provided the generated annotations and semantice pictures for training.
 Below shows the process of generating these files.

## Install pycococreatortools
```shell
sudo apt-get install python3-dev
pip install cython
pip install git+git://github.com/waspinator/coco.git@2.1.0
```

In [2]:
from pycococreatortools import pycococreatortools

## Split train and val datasets for final phase

In [8]:
!mkdir ./datasets/trainval/nu_seg #for semantic masks 
!mkdir ./datasets/trainval/cell_seg #for semantic masks 

mkdir: 无法创建目录"./datasets/trainval/nu_seg": 文件已存在
mkdir: 无法创建目录"./datasets/trainval/cell_seg": 文件已存在


In [9]:
img_dir = './datasets/trainval/x'
masks_dir = './datasets/trainval/y'
nu_seg_dir = './datasets/trainval/nu_seg'
cell_seg_dir = './datasets/trainval/cell_seg'

In [4]:
import os

In [14]:
img_ids = sorted(os.listdir(img_dir))
masks_ids = sorted(os.listdir(masks_dir))

In [15]:
train_inst = dict()
val_inst = dict()
for i,img_filename in enumerate(img_ids):
   # print(img_filename)
    img_id, _ = img_filename.split('.')
    if i < 448:
        train_inst[img_id] = []
    else:
        val_inst[img_id]=[]

In [16]:
print('train images nums:',len(train_inst))
print('val images nums:',len(val_inst))

train images nums: 448
val images nums: 50


In [17]:
for m in masks_ids:
    img_id, _ = m.split('_')
    if img_id in train_inst:
        train_inst[img_id].append(m)
    elif img_id in val_inst:
        val_inst[img_id].append(m)

In [18]:
train_inst

{'102': ['102_1.bmp',
  '102_2.bmp',
  '102_3.bmp',
  '102_4.bmp',
  '102_5.bmp',
  '102_6.bmp',
  '102_7.bmp',
  '102_8.bmp'],
 '104': ['104_1.bmp', '104_2.bmp', '104_3.bmp', '104_4.bmp'],
 '106': ['106_1.bmp', '106_2.bmp', '106_3.bmp', '106_4.bmp', '106_5.bmp'],
 '108': ['108_1.bmp', '108_2.bmp', '108_3.bmp', '108_4.bmp'],
 '109': ['109_1.bmp', '109_2.bmp', '109_3.bmp', '109_4.bmp', '109_5.bmp'],
 '110': ['110_1.bmp', '110_2.bmp', '110_3.bmp', '110_4.bmp'],
 '111': ['111_1.bmp', '111_2.bmp'],
 '112': ['112_1.bmp', '112_2.bmp', '112_3.bmp', '112_4.bmp'],
 '113': ['113_1.bmp', '113_2.bmp', '113_3.bmp'],
 '114': ['114_1.bmp', '114_2.bmp', '114_3.bmp'],
 '115': ['115_1.bmp', '115_2.bmp', '115_3.bmp', '115_4.bmp'],
 '1693': ['1693_2.bmp'],
 '1694': ['1694_1.bmp',
  '1694_2.bmp',
  '1694_3.bmp',
  '1694_4.bmp',
  '1694_5.bmp',
  '1694_6.bmp',
  '1694_7.bmp'],
 '1697': ['1697_1.bmp',
  '1697_10.bmp',
  '1697_11.bmp',
  '1697_2.bmp',
  '1697_3.bmp',
  '1697_5.bmp',
  '1697_6.bmp',
  '1697_7.

In [19]:
cnt1 = 0
cnt2 = 0
for m1 in train_inst:
    cnt1+= len(train_inst[m1])
for m2 in val_inst:
    cnt2+= len(val_inst[m2])
print('train masks nums:',cnt1)
print('val masks nums:',cnt2)

train masks nums: 2423
val masks nums: 210


## Get train and val annotations

In [20]:
import datetime
import cv2
import matplotlib.pylab as plt
import numpy as np

INFO = {
    "description": "SegPC2021 Dataset",
    "url": "https://github.com/waspinator/pycococreator",
    "contributor": "qiuxinyun",
}

CATEGORIES = [
    {
        'id': 1,
        'name': 'cell',
    },
 
]

train_nu_ann = {
    "info": INFO,
    "licenses": [],
    "categories": CATEGORIES,
    "images": [],
    "annotations": []
}

train_cell_ann= {
    "info": INFO,
    "licenses": [],
    "categories": CATEGORIES,
    "images": [],
    "annotations": []
}
val_nu_ann= {
    "info": INFO,
    "licenses": [],
    "categories": CATEGORIES,
    "images": [],
    "annotations": []
}
val_cell_ann= {
    "info": INFO,
    "licenses": [],
    "categories": CATEGORIES,
    "images": [],
    "annotations": []
}

In [21]:
def get_ann_output(insts, ann_output, seg_dir, flag='cell',vis=True):
    seg_id = 1
    nu_seg_temp=[]
    cell_seg_temp = []

    for i, img_id in enumerate(img_ids):
        img_key,_ = img_id.split('.')
        if  img_key not in insts:continue
        print(i,img_id)    
        img = cv2.imread(os.path.join(img_dir,img_id))
        imgsize = (img.shape[1],img.shape[0])
        image_info = pycococreatortools.create_image_info(
            i, os.path.basename(img_id), imgsize)
        ann_output["images"].append(image_info)
        
        seg_temp = []

        for mask_id in insts[img_key]:
            print('train:',mask_id)
           # img_key,_ = mask_id.split('_')
            mask = cv2.imread(os.path.join(masks_dir,mask_id),0)          
            if flag =='cell':
                mask = np.where(mask > 0 , 1, 0)
                seg_temp.append(mask)
            elif flag =='nu':
                mask = np.where(mask == 40 , 1, 0)
                seg_temp.append(mask)
            else:
                raise('error flag')
            category_info = {'id': 1, 'is_crowd': 0}
            ann_info = pycococreatortools.create_annotation_info(seg_id, i, category_info, mask, imgsize, tolerance=1)
            ann_output['annotations'].append(ann_info)
            seg_id+=1

        if seg_temp == []:
            seg = np.zeros(imgsize)
        else:
            seg = np.where(np.array(seg_temp).sum(0) > 0 ,1 ,0)
          


        cv2.imwrite(os.path.join(seg_dir,img_id),seg)

        if vis:
            f,ax = plt.subplots(1,2,figsize=(10,10))
            ax[0].imshow(img)
            ax[1].imshow(seg)
            plt.show()



In [ ]:
get_ann_output(val_inst, val_cell_ann, cell_seg_dir)
get_ann_output(val_inst, val_nu_ann, nu_seg_dir,flag='nu')

In [ ]:
get_ann_output(train_inst, train_cell_ann, cell_seg_dir,vis=False)
get_ann_output(train_inst, train_nu_ann, nu_seg_dir,flag='nu',vis=False)

In [ ]:
# check semantic picture
for img_id in img_ids:
    img_key,_ = img_id.split('.')
    if img_key not in val_inst:continue
    cell_seg = cv2.imread(os.path.join(cell_seg_dir,img_id),0)
    img = cv2.imread(os.path.join(img_dir,img_id))
    nu_seg = cv2.imread(os.path.join(nu_seg_dir,img_id),0)
    f,ax = plt.subplots(1,2,figsize=(10,10))
    print(img_id)
    ax[0].imshow(cell_seg+nu_seg)
    ax[1].imshow(img)
    plt.show()
    

In [24]:
import json

with open('./datasets/trainval/val_cell_t50.json', 'w') as f1:
    json.dump(val_cell_ann, f1)
with open('./datasets/trainval/val_nu_t50.json', 'w') as f2:
    json.dump(val_nu_ann, f2)   
with open('./datasets/trainval/train_cell_t448.json', 'w') as f3:
    json.dump(train_cell_ann, f3)    
with open('./datasets/trainval/train_nu_t448.json', 'w') as f4:
    json.dump(train_nu_ann, f4)    

## Check train and val annotations

In [25]:
from pycocotools.coco import COCO

In [ ]:
train_nu_coco = COCO('./datasets/trainval/train_nu_t448.json')
train_cell_coco = COCO('./datasets/trainval/train_cell_t448.json')
val_nu_coco = COCO('./datasets/trainval/val_nu_t50.json')
val_cell_coco = COCO('./datasets/trainval/val_cell_t50.json')

In [ ]:
for i,img_id in enumerate(img_ids):
    img_key,_ = img_id.split('.')
    img = cv2.imread(os.path.join(img_dir,img_id))
    if img_key in train_inst:
        print(i,'train:',img_id)
        nu_ann_ids = train_nu_coco.getAnnIds(imgIds=i)
        nu_anns = train_nu_coco.loadAnns(nu_ann_ids)
        cell_ann_ids = train_cell_coco.getAnnIds(imgIds=i)
        cell_anns = train_cell_coco.loadAnns(cell_ann_ids) 
    elif img_key in val_inst:
        print(i,'val:',img_id)
        nu_ann_ids = val_nu_coco.getAnnIds(imgIds=i)
        nu_anns = val_nu_coco.loadAnns(nu_ann_ids)
        cell_ann_ids = val_cell_coco.getAnnIds(imgIds=i)
        cell_anns = val_cell_coco.loadAnns(cell_ann_ids) 
          
    temp = []
    for nu_ann,cell_ann in zip(nu_anns,cell_anns):
        if img_key in train_inst:
            nu_mask = train_nu_coco.annToMask(nu_ann)
            cell_mask = train_cell_coco.annToMask(cell_ann)
        elif img_key in val_inst:
            nu_mask = val_nu_coco.annToMask(nu_ann)
            cell_mask = val_cell_coco.annToMask(cell_ann)            
        inst = nu_mask + cell_mask
        temp.append(inst)
    if temp ==[]:
        temp = np.zeros_like(img)
    seg = np.array(temp).sum(0)
    f,ax = plt.subplots(1,2,figsize=(10,10))
    ax[0].imshow(seg)
    ax[1].imshow(img)
    plt.show()

## Get test annotations

In [36]:
test_ann= {
    "info": INFO,
    "licenses": [],
    "categories": CATEGORIES,
    "images": [],
    "annotations": []
}

In [32]:
test_img_dir  = "./datasets/test/x"
test_img_ids = sorted(os.listdir(test_img_dir))

In [34]:
len(test_img_ids)

277

In [37]:
for test_img_id,img_name in enumerate(test_img_ids):
    print(test_img_id,img_name)
    image = plt.imread(os.path.join(test_img_dir,img_name))
    imagesize = (image.shape[1],image.shape[0])
    image_info = pycococreatortools.create_image_info(
        test_img_id, os.path.basename(img_name), imagesize)
    test_ann["images"].append(image_info)

0 101.bmp
1 103.bmp
2 105.bmp
3 107.bmp
4 1703.bmp
5 1707.bmp
6 1710.bmp
7 1711.bmp
8 1723.bmp
9 1729.bmp
10 1735.bmp
11 1754.bmp
12 1760.bmp
13 1761.bmp
14 1763.bmp
15 1768.bmp
16 1784.bmp
17 1785.bmp
18 1793.bmp
19 1794.bmp
20 1796.bmp
21 1797.bmp
22 1804.bmp
23 1806.bmp
24 1820.bmp
25 1827.bmp
26 1828.bmp
27 1836.bmp
28 1840.bmp
29 1850.bmp
30 1852.bmp
31 1868.bmp
32 1879.bmp
33 1884.bmp
34 1887.bmp
35 1889.bmp
36 1892.bmp
37 1893.bmp
38 1903.bmp
39 1910.bmp
40 1922.bmp
41 1923.bmp
42 1924.bmp
43 1927.bmp
44 1938.bmp
45 1945.bmp
46 1947.bmp
47 1950.bmp
48 1953.bmp
49 1968.bmp
50 1969.bmp
51 1974.bmp
52 1975.bmp
53 1976.bmp
54 1986.bmp
55 1988.bmp
56 1995.bmp
57 2006.bmp
58 2014.bmp
59 2016.bmp
60 2028.bmp
61 2030.bmp
62 204.bmp
63 205.bmp
64 2061.bmp
65 2065.bmp
66 2077.bmp
67 2079.bmp
68 2084.bmp
69 209.bmp
70 2102.bmp
71 2103.bmp
72 211.bmp
73 2113.bmp
74 2114.bmp
75 2115.bmp
76 2119.bmp
77 2120.bmp
78 2123.bmp
79 2124.bmp
80 2126.bmp
81 2127.bmp
82 2131.bmp
83 2133.bmp
84 2134.bm

In [39]:
test_ann_save_path = "./datasets/test/test.json"
with open(test_ann_save_path, 'w') as test_ann_file:
    json.dump(test_ann, test_ann_file)